In [1]:
import numpy as np
import math
import statistics

def afinize(P):
    x = []
    x.append(P[0]/P[2])
    x.append(P[1]/P[2])
    return x

def dist(a, b):
    n = len(a)
    d = 0
    for i in range(n):
        d = d + (b[i] - a[i]) ** 2
    return math.sqrt(d)

import matplotlib.pyplot as plt 
#za crtanje tacki  
def nacrtaj_grafik(tacke, naslov):
    x = [x[0] for x in tacke] 
    y = [y[1] for y in tacke] 

    plt.scatter(x, y, label = naslov, color= "green",  
                marker= "o", s=30) 

    plt.xlabel('x - osa') 
    plt.ylabel('y - osa') 
    plt.title('Tacke')  
    plt.legend()
    
    plt.show() 

In [2]:
def matrica_preslikavanja(a, b, c, d):
    
    A = np.stack((a, b, c), axis = -1)
    B = np.array(d)
    
    koef = np.linalg.inv(A).dot(B)
    
    aa = np.array(a) * koef[0]
    bb = np.array(b) * koef[1]
    cc = np.array(c) * koef[2]
    
    P = np.stack((aa, bb, cc), axis = -1)
    
    return P

In [3]:
def naivni(original, slika):
    [a, b, c, d] = original
    [ap, bp, cp, dp] = slika
    
    P1 = matrica_preslikavanja(a, b, c, d)
    P2 = matrica_preslikavanja(ap, bp, cp, dp)
    P = np.dot(P2, np.linalg.inv(P1))
    
    return P

In [4]:
def matrica_2x9(m, mp):
    M = np.zeros((2,9))
    
    M[1][0] = mp[2] * m[0]
    M[1][1] = mp[2] * m[1]
    M[1][2] = mp[2] * m[2]
    
    M[0][3] = -mp[2] * m[0]
    M[0][4] = -mp[2] * m[1]
    M[0][5] = -mp[2] * m[2]
    
    M[0][6] = mp[1] * m[0]
    M[0][7] = mp[1] * m[1]
    M[0][8] = mp[1] * m[2]
    
    M[1][6] = -mp[0] * m[0]
    M[1][7] = -mp[0] * m[1]
    M[1][8] = -mp[0] * m[2]
    
    return M

In [5]:
def dlt(original, slika):
    n = min(len(original), len(slika))
    niz_mi = []
    
    for i in range(n):
        Mi = matrica_2x9(original[i], slika[i])
        niz_mi.append(Mi)
          
    M = np.stack(niz_mi)
    M = M.reshape(2*n, 9)
    
    _, _, ut = np.linalg.svd(M)
    P = ut[-1]
    P = P.reshape(3, 3)
    
    return P

In [6]:
def matrica_normalizacije(tacke):

    afine_tacke = [afinize(x) for x in tacke]
    
    x = sum([x[0] for x in afine_tacke]) / len(afine_tacke)
    y = sum([x[1] for x in afine_tacke]) / len(afine_tacke)
    C = [x, y]
    #teziste
    
    S = np.diagflat([1.0, 1.0, 1.0])
    G = np.diagflat([1.0, 1.0, 1.0])
    
    #matrica translacije
    G[0][2] = -C[0]
    G[1][2] = -C[1]
    
    #translirane tacke za vektor
    afine_tacke_trans = [[x-C[0], y-C[1]] for [x, y] in afine_tacke]
    
    #racunanje prosecne udaljenosti on koordinatnog pocetka
    avg_dist = statistics.mean([dist(x, [0,0]) for x in afine_tacke_trans])
    
    #matrica skaliranja
    S[0][0] = (math.sqrt(2)) / avg_dist
    S[1][1] = (math.sqrt(2)) / avg_dist
    
    T = np.matmul(S, G)
    
    print("Matrica normalizacije:\n", T, "\n")
    return T

In [7]:
#funkcija koja transformise niz kolona u niz redova
def vektor_redovi(matrix):
    
    row_matrix = []
    
    for row in matrix:
        
        new_row = []
        
        for column in row:
            [val] = column
            new_row.append(val)
        
        row_matrix.append(new_row)
    
    return row_matrix

In [8]:
def dlt_modifikovani(original, slika):
    
    T = matrica_normalizacije(original)
    Tp = matrica_normalizacije(slika)
    
    #nacrtaj_grafik([afinize(x) for x in original], "original")
    #nacrtaj_grafik([afinize(x) for x in slika], "slika")
    
    original_norm = [np.matmul(T, np.array(x).reshape(3, 1)) for x in original]
    slika_norm = [np.matmul(Tp, np.array(x).reshape(3, 1)) for x in slika]
    
    original_norm_row = vektor_redovi(original_norm)
    slika_norm_row = vektor_redovi(slika_norm)
    
    Pn = dlt(original_norm_row, slika_norm_row)
    
    P = np.dot(np.linalg.inv(Tp), np.dot(Pn, T))
    
    return P

In [9]:
# a = [-3, -1, 1]
# b = [3, -1, 1]
# c = [1, 1, 1]
# d = [-1, 1, 1]

# ap = [-2, -1, 1]
# bp = [2, -1, 1]
# cp = [2, 1, 1]
# dp = [-2, 1, 1]

a = [-3, 2, 1]
b = [-2, 5, 2]
c = [1, 0, 3]
d = [-7, 3, 1]

ap = [11, -12, 7]
bp = [25, -8, 9]
cp = [15, 4, 17]
dp = [14, -28, 10]

print("Algoritmi za 4 tacke:\n")
print("==============================================================================\n")

print([a, b, c, d], "\n")
print([ap, bp, cp, dp], "\n")

P_naivni = naivni([a, b, c, d], [ap, bp, cp, dp])

print("Matrica preslikavanja dobijena NAIVNIM algoritmom:\n", P_naivni, "\n")

P_naivni_zaokruzeno = np.around(P_naivni, 5)

print("Matrica preslikavanja dobijena DLT algoritmom, zaokruzena na 5 decimala:\n", P_naivni_zaokruzeno, "\n")

P_dlt = dlt([a, b, c, d], [ap, bp, cp, dp])

print("Matrica preslikavanja dobijena DLT algoritmom:\n", P_dlt, "\n")

P_dlt_zaokruzeno = np.around(P_dlt, 5)

print("Matrica preslikavanja dobijena DLT algoritmom, zaokruzena na 5 decimala:\n", P_dlt_zaokruzeno, "\n")

P_dlt_poredjenje = (P_dlt_zaokruzeno / P_dlt_zaokruzeno[0][1]) * P_naivni[0][1]

print("Matrica preslikavanja dobijena DLT algoritmom, za poredjenje sa naivnim:\n", P_dlt_poredjenje, "\n")

P_dlt_mod = dlt_modifikovani([a, b, c, d], [ap, bp, cp ,dp])

print("Matrica preslikavanja dobijena modifikovanim DLT algoritmom:\n", P_dlt_mod, "\n")

P_dlt_mod_zaokruzeno = np.round(P_dlt_mod, 5)

print("Matrica preslikavanja dobijena modifikovanim DLT algoritmom, zaokruzena na 5 decimala:\n", P_dlt_mod_zaokruzeno, "\n")

P_dlt_mod_poredjenje = np.round((P_dlt_mod_zaokruzeno / P_dlt_mod_zaokruzeno[0][1]) * P_naivni[0][1])

print("Matrica preslikavanja dobijena modifikovanim DLT algoritmom, za poredjenje sa naivnim:\n", P_dlt_mod_poredjenje, "\n")

print("==============================================================================\n")

Algoritmi za 4 tacke:


[[-3, 2, 1], [-2, 5, 2], [1, 0, 3], [-7, 3, 1]] 

[[11, -12, 7], [25, -8, 9], [15, 4, 17], [14, -28, 10]] 

Matrica preslikavanja dobijena NAIVNIM algoritmom:
 [[ 3.68006279e-15  3.00000000e+00  5.00000000e+00]
 [ 4.00000000e+00 -1.05471187e-15 -7.90217565e-16]
 [-1.00000000e+00 -1.00000000e+00  6.00000000e+00]] 

Matrica preslikavanja dobijena DLT algoritmom, zaokruzena na 5 decimala:
 [[ 0.  3.  5.]
 [ 4. -0. -0.]
 [-1. -1.  6.]] 

Matrica preslikavanja dobijena DLT algoritmom:
 [[ 3.33066907e-16  3.19801075e-01  5.33001791e-01]
 [ 4.26401433e-01 -5.06038675e-16 -1.21180353e-16]
 [-1.06600358e-01 -1.06600358e-01  6.39602149e-01]] 

Matrica preslikavanja dobijena DLT algoritmom, zaokruzena na 5 decimala:
 [[ 0.      0.3198  0.533 ]
 [ 0.4264 -0.     -0.    ]
 [-0.1066 -0.1066  0.6396]] 

Matrica preslikavanja dobijena DLT algoritmom, za poredjenje sa naivnim:
 [[ 0.  3.  5.]
 [ 4. -0. -0.]
 [-1. -1.  6.]] 

Matrica normalizacije:
 [[ 0.55728547  0.          1.4

In [12]:
print("Model za odbranu domaceg zadatka:\n")
print("==============================================================================\n")
#originalne tacke
a = [-3, 2, 1]
b = [-2, 5, 2]
c = [1, 0, 3]
d = [-7, 3, 1]
e = [2, 1, 2]
f = [-1, 2, 1]
g = [1, 1, 1]
#slike tacaka
ap = [11, -12, 7]
bp = [25, -8, 9]
cp = [15, 4, 17]
dp = [14, -28, 10]
ep = [13, 8, 9]
fp = [11, -4, 5]
gp = [8.02, 4, 4]
#pokvarili smo sliku tacke g na drugoj decimali

originali = [a, b, c, d, e, f, g]
slike = [ap, bp, cp, dp, ep, fp, gp]

print(originali, "\n")
print(slike, "\n")

P_naivni = naivni([a, b, c, d], [ap, bp, cp, dp])
P_naivni_zaokruzeno = np.round(P_naivni)
print("NAIVNI algoritam za 4 tacke, zaokruzen na ceo broj:\n", P_naivni_zaokruzeno, "\n")

P_dlt = np.round(dlt(originali, slike), 6)
print("DLT algoritam:\n", P_dlt, "\n")

P_dlt_poredjenje = np.round(P_dlt / P_dlt[0][1] * 3, 2)
print("DLT algoritam za poredjenje sa naivnim:\n", P_dlt_poredjenje, "\n")
#slazu se do na dve decimale, jer smo pokvarili tacke za dlt

P_ndlt = np.round(dlt_modifikovani(originali, slike), 6)
print("Modifikovani DLT algoritam:\n", P_ndlt, "\n")

P_ndlt_poredjenje = np.round(P_ndlt / P_ndlt[0][1] * 3, 2)
print("DLT algoritam za poredjenje sa naivnim:\n", P_ndlt_poredjenje, "\n")

print("==============================================================================\n")

Model za odbranu domaceg zadatka:


[[-3, 2, 1], [-2, 5, 2], [1, 0, 3], [-7, 3, 1], [2, 1, 2], [-1, 2, 1], [1, 1, 1]] 

[[11, -12, 7], [25, -8, 9], [15, 4, 17], [14, -28, 10], [13, 8, 9], [11, -4, 5], [8.02, 4, 4]] 

NAIVNI algoritam za 4 tacke, zaokruzen na ceo broj:
 [[ 0.  3.  5.]
 [ 4. -0. -0.]
 [-1. -1.  6.]] 

DLT algoritam:
 [[ 5.30000e-05  3.20011e-01  5.32929e-01]
 [ 4.26446e-01 -2.00000e-05 -3.00000e-05]
 [-1.06595e-01 -1.06519e-01  6.39542e-01]] 

DLT algoritam za poredjenje sa naivnim:
 [[ 0.  3.  5.]
 [ 4. -0. -0.]
 [-1. -1.  6.]] 

Matrica normalizacije:
 [[ 0.60235224  0.          0.83181976]
 [ 0.          0.60235224 -0.94655352]
 [ 0.          0.          1.        ]] 

Matrica normalizacije:
 [[ 1.06933585  0.         -1.87607394]
 [ 0.          1.06933585  0.62311599]
 [ 0.          0.          1.        ]] 

Modifikovani DLT algoritam:
 [[ 3.90000e-05  2.34697e-01  3.90851e-01]
 [ 3.12756e-01 -1.50000e-05 -2.20000e-05]
 [-7.81770e-02 -7.81210e-02  4.69042e-01]] 

DL

In [13]:
print("Testiranje invarijantnosti u odnosu na promenu koordinata:\n")
print("==============================================================================\n")

CC = np.array([[0, 1, 2], [-1, 0, 3], [0, 0, 1]])
print("Matrica promene koordinata:\n", CC, "\n")

n_originali_kol = [np.matmul(CC, np.array(x).reshape(3,1)) for x in originali]
n_originali = vektor_redovi(n_originali_kol)
print("Koordinate originala u novom koordinatnom sistemu:\n", n_originali, "\n")

n_slike_kol = [np.matmul(CC, np.array(x).reshape(3,1)) for x in slike]
n_slike = vektor_redovi(n_slike_kol)
print("Koordinate slika u novom koordinatnom sistemu:\n", n_slike, "\n")

#odrediti matricu preslikavanja u novom koord sistemu
M_dlt = dlt(n_originali, n_slike)
#vracanje u stari
M_dlt_s = np.dot(np.linalg.inv(CC), np.dot(M_dlt, CC))
print("Matrica preslikavanja u starom koordinatnom sistemu\n", M_dlt_s, "\n")

M_dlt_s_skalirano = np.round(M_dlt_s / M_dlt_s[0][1] * 3, 3)
print("Matrica preslikavanja u starom koordinatnom sistemu, skalirana:\n", M_dlt_s_skalirano, "\n")

print("Izracunata matrica pre promene koordinata, radi poredjenja:\n", np.round(P_dlt / P_dlt[0][1] * 3, 3), "\n")

#modifikovani
#odrediti matricu preslikavanja u novom koord sistemu
M_dlt_mod = dlt_modifikovani(n_originali, n_slike)
#vracanje u stari
M_dlt_mod_s = np.dot(np.linalg.inv(CC), np.dot(M_dlt_mod, CC))
print("Matrica preslikavanja u starom koordinatnom sistemu\n", M_dlt_mod_s, "\n")

M_dlt_mod_s_skalirano = np.round(M_dlt_mod_s / M_dlt_mod_s[0][1] * 3, 3)
print("Matrica preslikavanja u starom koordinatnom sistemu, skalirana:\n", M_dlt_mod_s_skalirano, "\n")

print("Izracunata matrica pre promene koordinata, radi poredjenja:\n", np.round(P_ndlt / P_ndlt[0][1] * 3, 3), "\n")


print("==============================================================================\n")

Testiranje invarijantnosti u odnosu na promenu koordinata:


Matrica promene koordinata:
 [[ 0  1  2]
 [-1  0  3]
 [ 0  0  1]] 

Koordinate originala u novom koordinatnom sistemu:
 [[4, 6, 1], [9, 8, 2], [6, 8, 3], [5, 10, 1], [5, 4, 2], [4, 4, 1], [3, 2, 1]] 

Koordinate slika u novom koordinatnom sistemu:
 [[2, 10, 7], [10, 2, 9], [38, 36, 17], [-8, 16, 10], [26, 14, 9], [6, 4, 5], [12.0, 3.9800000000000004, 4.0]] 

Matrica preslikavanja u starom koordinatnom sistemu
 [[-1.73564676e-05 -1.04847554e-01 -1.74607666e-01]
 [-1.39719503e-01  6.61760914e-06  9.63372330e-06]
 [ 3.49246285e-02  3.48997143e-02 -2.09538156e-01]] 

Matrica preslikavanja u starom koordinatnom sistemu, skalirana:
 [[ 0.     3.     4.996]
 [ 3.998 -0.    -0.   ]
 [-0.999 -0.999  5.996]] 

Izracunata matrica pre promene koordinata, radi poredjenja:
 [[ 0.     3.     4.996]
 [ 3.998 -0.    -0.   ]
 [-0.999 -0.999  5.996]] 

Matrica normalizacije:
 [[ 0.60235224  0.         -2.15125801]
 [ 0.          0.60235224 -2.6